In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test= np.load("/content/drive/Shareddrives/2022동계우수학부생_DSAIL/chest_xray_data_rgb.npy", allow_pickle=True)

In [ ]:
resnet_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=X_train[0].shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)
resnet_model.trainable = True

resnet_model.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [ ]:
for layer in resnet_model.layers[:-4]:
   layer.trainable = False

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(resnet_model.output)
x = tf.keras.layers.Flatten()(resnet_model.output)
# x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(resnet_model.input, x)

In [ ]:
# augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest')

train_generator = datagen.flow(X_train, y_train,batch_size=32,shuffle=True,seed=42)
valid_generator = datagen.flow(X_val,y_val,batch_size=32,seed=42)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss='binary_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_val, y_val))
history = model.fit(train_generator,epochs=15, validation_data=valid_generator) # augmentation 적용

results = model.evaluate(X_test, y_test)
print(results)

Epoch 1/15
163/163 [==============================] - 84s 429ms/step - loss: 2.0078 - accuracy: 0.9179 - val_loss: 2.0678 - val_accuracy: 0.6250
Epoch 2/15
163/163 [==============================] - 68s 418ms/step - loss: 0.1600 - accuracy: 0.9442 - val_loss: 0.8079 - val_accuracy: 0.8125
Epoch 3/15
163/163 [==============================] - 68s 418ms/step - loss: 0.1443 - accuracy: 0.9507 - val_loss: 0.7620 - val_accuracy: 0.8125
Epoch 4/15
163/163 [==============================] - 68s 417ms/step - loss: 0.1210 - accuracy: 0.9532 - val_loss: 0.8907 - val_accuracy: 0.5625
Epoch 5/15
163/163 [==============================] - 69s 422ms/step - loss: 0.1131 - accuracy: 0.9555 - val_loss: 0.8154 - val_accuracy: 0.7500
Epoch 6/15
163/163 [==============================] - 69s 420ms/step - loss: 0.1252 - accuracy: 0.9549 - val_loss: 0.7681 - val_accuracy: 0.6250
Epoch 7/15
163/163 [==============================] - 69s 423ms/step - loss: 0.1013 - accuracy: 0.9636 - val_loss: 0.2526 - val_ac

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.reshape(624)
y_pred1 = np.zeros(624)
for i in range(len(y_pred)):
  if y_pred[i] > 0.5:
    y_pred1[i]=1


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
target_names = ['NORMAL', 'PNEUMONIA']
print(confusion_matrix(y_test, y_pred1))
print(classification_report(y_test, y_pred1, target_names=target_names))

[[212  22]
 [ 15 375]]
              precision    recall  f1-score   support

      NORMAL       0.93      0.91      0.92       234
   PNEUMONIA       0.94      0.96      0.95       390

    accuracy                           0.94       624
   macro avg       0.94      0.93      0.94       624
weighted avg       0.94      0.94      0.94       624

